In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
model = ModelfClassification(10)
criterion = nn.MSELoss()  # Используем RMSE как функцию потери
# Определение оптимизатора
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

C:\Users\fktrc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        sample = {'x': self.x[idx], 'y': self.y[idx]}
        return sample

# Загрузка данных из файлов .npy
y_smp_train = np.load('y_smp_train.npy')
pars_smp_train = np.load('pars_smp_train.npy')

# # Разделение данных на тренировочные и тестовые выборки
# X_train, X_test, y_train, y_test = train_test_split(y_smp_train, pars_smp_train, test_size=0.2, random_state=42)

# # Преобразование данных в тензоры PyTorch и перемещение на GPU
# X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
# y_train = torch.tensor(y_train, dtype=torch.float32).to(device)

# X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
# y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

y_smp_train = torch.tensor(y_smp_train, dtype=torch.float32).to(device)
pars_smp_train = torch.tensor(pars_smp_train, dtype=torch.float32).to(device)

pars_smp_train = pars_smp_train.reshape(pars_smp_train.shape[0], -1)

# Создание экземпляра CustomDataset для тренировочных и тестовых данных
# train_dataset = CustomDataset(X_train, y_train)
# test_dataset = CustomDataset(X_test, y_test)
full_dataset = CustomDataset(y_smp_train, pars_smp_train)

# Создание DataLoader для тренировочных и тестовых данных
batch_size = 20
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size)
full_loader = DataLoader(full_dataset, batch_size=batch_size)


NameError: name 'Dataset' is not defined

In [7]:
def train_model(model, train_loader, criterion, optimizer, num_epochs, scheduler):
    model.train()
    train_losses = []

    for epoch in range(num_epochs):
        epoch_losses = []
        for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):  # Используем tqdm
            input_data, target_data = batch['x'], batch['y'].reshape(batch['y'].shape[0], 15)
            output = model(input_data)
            loss = criterion(output, target_data)
            loss = torch.sqrt(loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_losses.append(loss.item())

        scheduler.step()
        # Отображение графика потерь после каждого батча
        plt.plot(epoch_losses)
        plt.xlabel('Batch')
        plt.ylabel('Loss')
        plt.title('Training Loss for Batch')
        plt.show()

        train_losses.extend(epoch_losses)

    return train_losses


In [8]:
train_model(model, full_loader, criterion, optimizer, 50, scheduler)

NameError: name 'full_loader' is not defined